### Lets read the results

In [3]:
import pandas as pd
import re
import json
from collections import Counter
from nltk.corpus import stopwords
import nltk

import emoji
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize

import os

In [27]:
RemoveStops(three, 'english')

'pizza hut favorite restaurant'

In [16]:
all_words = []
one = 'hello i love pizza'
two = 'where is my pizza delivery guy'
three = 'pizza hut is my favorite restaurant'


    
for sentence in [one,two,three]:
    cleaned_sentence = RemoveStops(sentence, 'english')    
    for word in cleaned_sentence.split(' '):
        all_words.append(word)

In [28]:
pd.DataFrame({ 'doc_one' : {'delivery' : 0, 'favorite' : 0, 'guy' : 0, 'hello' : 1, 'hut' : 0, 'love': 1, 'pizza' : 1, 'restaurant' : 0},
 'doc_two' : {'delivery' : 1, 'favorite' : 0, 'guy' : 1, 'hello' : 0, 'hut' : 0, 'love': 0, 'pizza' : 1 , 'restaurant' : 0},
 'doc_three' : {'delivery' : 0, 'favorite' : 1, 'guy' : 0, 'hello' : 0, 'hut' : 1, 'love' : 0, 'pizza': 1, 'restaurant' : 1} }).transpose()

,delivery,favorite,guy,hello,hut,love,pizza,restaurant
doc_one,0,0,0,1,0,1,1,0
doc_two,1,0,1,0,0,0,1,0
doc_three,0,1,0,0,1,0,1,1


In [4]:
def folderCompProfiles(comp_name):
    """Searches in the data folder to search for followerProfiles on the name of this company."""
    #Get all the followerProfiles
    followerProfiles = [value for value in os.listdir("insta_followerprofile/data") if 'followerProfile' in value]
    company_followerProfiles = [value for value in followerProfiles if comp_name + '_' in value]
    return company_followerProfiles

folderCompProfiles('voetbaldirect')

['voetbaldirect_followerProfile.csv',
 'voetbaldirect_second_followerProfile.csv']

In [5]:
def mergeRawDataFrame(file_locations):
    """Takes as input a list of file locations. First checks the usernames present in all files and notifies
    the user if there is an overlap. After this it merges all the follower profiles."""
    df = pd.DataFrame()
    name_buffer = {}
    for followerProfile in file_locations:
        temp_df = pd.read_csv("../data/rawdata/" + followerProfile)
        names = [value for value in set([value for value in temp_df['data.user.edge_owner_to_timeline_media.edges.node.owner.username']])]
        #Small check to see whether names are present in different followerProfiles of the same company.
        for name in names:
            if name in name_buffer.keys():
                print('We have a double name: {}'.format(name))
                name_buffer[name] += 1
            else:
                name_buffer[name] = 1
        df = pd.concat([df, temp_df], ignore_index=True)
    return df

In [6]:
## PLEASE READ: ONLY READ THIS WHEN USING THE NEW WORKFLOW OF STEVESIE DATA. OTHERWISE THE TEMPLATE IS DIFFERENT!

def cleanRawDataFrame(raw_dataframe):
    """Select the desired columns from the raw dataframe and renames the columns."""
    cleaned_df = raw_dataframe[[
         'input.user_id', 
         'data.user.edge_owner_to_timeline_media.edges.node.owner.username', 
         'data.user.edge_owner_to_timeline_media.edges.node.shortcode', 
         'data.user.edge_owner_to_timeline_media.edges.node.edge_media_to_caption.edges[0].node.text', 
         'data.user.edge_owner_to_timeline_media.edges.node.display_url', 
         'data.user.edge_owner_to_timeline_media.edges.node.video_url', 
         'data.user.edge_owner_to_timeline_media.edges.node.edge_media_preview_like.count', 
         'data.user.edge_owner_to_timeline_media.edges.node.edge_media_to_comment.count', 
         'data.user.edge_owner_to_timeline_media.edges.node.video_view_count', 
         'data.user.edge_owner_to_timeline_media.edges.node.taken_at_timestamp']]
    
    new_columnNames = ['userID',
                      'username',
                      'shortcode',
                      'original_text',
                      'display_url',
                      'video_url',
                      'like_count',
                      'comment_count',
                       'view_count',
                      'timestamp']
    
    cleaned_df.columns = new_columnNames
    return cleaned_df

### Create seperate column which indicate occurance of hashtags

In [4]:
from collections import Counter
def cleanUnicode(string):
    """Some simple cleanup applied to a string of text"""
    string = string.lower() #No capital letters
    string = string.replace('\n',' ') # Remove new line commands
    string = string.replace('\xa0',' ') # Remove non-breaking space in Latin
    return string
    
    
def removeHashtags(string):
    """Takes string as input. Removes all words which start with # with length > 2"""
    sentence_words = string.split(' ')
    new_sentence = []
    for word in sentence_words:
        if word[0] == '#' and len(word) > 2:
            continue
        else:
            new_sentence.append(word)
    return ' '.join(new_sentence)
            
    
def extractHashtags(string):
    '''This function simply takes a string as input and analyses all words that start with a hashtag.'''
    buffer = []
    try:
        for value in string.split(' '):
            try:
                if value[0] == '#':
                    #Counter the number of hashtags in the hashtag. If > 0 we need to further split.
                    if dict(Counter(value))['#'] > 1:
                        multiple_hashtags = value.split('#')
                        for hashtag in multiple_hashtags:
                            if len(hashtag) > 2:
                                buffer.append('#' + hashtag)
                    #Only one hashtag in the word? Just add it to the buffer if length > 2.
                    else:
                        if len(value) > 2: buffer.append(value)
            except IndexError:
                continue
    except SyntaxError:
        print('oei')
    return buffer

def extractUser(string):
    '''This function simply takes a string as input and analyses all words that start with a hashtag.'''
    buffer = []
    try:
        for value in string.split(' '):
            try:
                if value[0] == '@':
                    #Counter the number of hashtags in the hashtag. If > 0 we need to further split.
                    if dict(Counter(value))['@'] > 1:
                        multiple_hashtags = value.split('@')
                        for hashtag in multiple_hashtags:
                            if len(hashtag) > 2:
                                buffer.append('@' + hashtag)
                    #Only one hashtag in the word? Just add it to the buffer if length > 2.
                    else:
                        if len(value) > 2: buffer.append(value)
            except IndexError:
                continue
    except SyntaxError:
        print('oei')
    return buffer

def deEmojify(inputString):
    return emoji.get_emoji_regexp().sub(u'', inputString)

def removePunct_ONEWORD(string):
    """This function uses a string of words as input and cleans it as good as it can."""
    string= string.replace('.', ' ')
    temp = re.sub(r'[^\w\s]','',string)
    return temp.replace(' ', '.')

def RemoveStops(string, lang):
    """This takes a string as input, splits it on the spaces and removes the stopwords."""
    stop_words = set(stopwords.words(lang)) 
    temp = [word for word in string.split(" ") if not word in stop_words]
    return ' '.join(temp)

def processText(original_string):
    #Remove capital and some weird unicode.
    string = cleanUnicode(original_string)
    #Extract user mentions and hashtags so we can remove this from the text.
    hashtags = extractHashtags(string)
    userMentions = extractUser(string)
    #Removing the hashtags and user mentions seperately from the list of words.
    string_noHashtags = [value for value in word_tokenize(string) if '#' + value not in hashtags]
    string_noUsers = [value for value in string_noHashtags if value[1::] not in userMentions]
    #Add all words together into a sentence if the word length > 1. (remove punc etc.)
    processed_text =  ' '.join([value for value in string_noUsers if len(value) > 1])
    #Finally remove the emoticons and we are done!
    hashtags_cleaned = [removePunct_ONEWORD(value) for value in hashtags]
    userMentions_cleaned = [removePunct_ONEWORD(value) for value in userMentions]
    #Now lets remove some dutch and English stopwords from the processed text.
    processed_text_noStops = RemoveStops(RemoveStops(processed_text, 'dutch'), 'english')
    #Finally remove the emoticons from the no stopwords text and we are done!
    processed_text_noEmoticons = deEmojify(processed_text_noStops)
    return [original_string, processed_text_noEmoticons, hashtags_cleaned, userMentions_cleaned]

In [8]:
def createProcessedSenteceDF(followerProfile_sentences):
    """Takes the written texts of the posts from a followerProfile as input. Extract hashtags and usernames to
    create three seperate columns in the dataframe to increase processing."""
    df_buffer = []
    for sentence in sentences:   
        #If the text is just NaN (float) we want to skip it.
        if isinstance(sentence, float) == True:
            df_buffer.append(['', '', '', ''])
        else:
            df_buffer.append(processText(sentence))

    return pd.DataFrame(df_buffer)

In [12]:
company_names = ["madedotcom", "vtwonen", "hemanederland", "loods5", "ikeanederland", "homify", "westwingnl", 
"karwei", "kwantum_nederland", "xenos_nl", "homedeco", "bol_com", "leenbakker", "wonenmetlef", "_connox_",
"interiorjunkiecom", "jysknl", "wehkamp", "fonqnl", "konforhome", "basiclabel.nl", "blokker",
"deensnl", "hastensbeds", "eijerkamp", "goossenswonenenslapen", "furn.nl", "stoermetaal", "roomednl", "misterdesignnl",
"dekbeddiscounter", "woonexpress", "zitmaxx", "pronto_wonen", "designbestseller", "barbecueshop.nl",
"flinders.design", "trendhopper", "debommelmeubelen", "otto_nl", "praxis_bouwmarkt", "gamma_nl",
"pietklerkx.nl", "swisssense", "montelwonen", "aupingnl", "hacowonenenslapen", "emma_matras", "hornbachnl",
"lampenlicht.nl", "profijtmeubel", "bianonl", "woonboulevardpoortvliet", "morreswonen", "hubo_nl", "beter_bed",
"hoogenboezem.meubelen", "villajipp_outlet", "vidaxl_nl", "mline_nl"]

food_companies = ["veganjunkfoodbar", "pastaebasta_amsterdam", "mamakellyamsterdam", "watsonsfood",
"cannibaleroyale", "parkheuvel", "restaurantfred", "hugh_rotterdam", "oldscuola", "restaurantkite",
"wturbankitchen", "thestreetfoodclub", "rumclubutrecht", "lejardinutrecht", "broei.utrecht"]

sport_companies = ["plutosport.nl", "voetbalshopnl", "all4runningstore", "voetbaldirect",
"dakasport", "hockeydirect.nl", "tennisdirect", "intersportnl", "aktiesport.nl", "sport2000nederland",
"soccerfanshop", "jdsportsnl", "decathlonnederland", "gorillasportsnl", "perrysport.nl"]

In [11]:
for comp_name in sport_companies:    
    #Find all company follower profiles in the data folder
    company_followerProfiles = folderCompProfiles(comp_name)
    #Merge all company follower profiles in one final dataframe
    cleaned_df = mergeRawDataFrame(company_followerProfiles)
    #Give the columns of the new dataframe the desired column names.
    cleaned_df = cleanRawDataFrame(cleaned_df)
    #Storing all the texts of all posts under one variable name
    sentences = [value for value in cleaned_df['original_text']]
    #Creating a new dataframe which contains all the processed posts
    cleaned_sentence_df = createProcessedSenteceDF(sentences)
    #Add these columns of the cleaned sentence dataframe to the cleaned dataframe we already have
    cleaned_df['cleaned_text'] = [value for value in cleaned_sentence_df[1]]
    cleaned_df['hashtags'] = [value for value in cleaned_sentence_df[2]]
    cleaned_df['userMsentions'] = [value for value in cleaned_sentence_df[3]]
    #Printing the total number of userIDs from which we extracted the posts now.
    total_userIDS = len(set([value for value in cleaned_df['userID']]))
    #Finally storing the final dataframe in the data folder.\
    print("Finished cleaning profile of {}. Total IDS: {}.".format(comp_name, total_userIDS))
    cleaned_df.to_pickle("../data/cleanedprofiles/{}_cleaned.pkl".format(comp_name))

<ipython-input-5-c29f81a2966e>:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([df, temp_df], ignore_index=True)


Finished cleaning profile of perrysport.nl. Total IDS: 2003.


In [13]:
def check_IDS(comp_name):
    #Find all company follower profiles in the data folder
    company_followerProfiles = folderCompProfiles(comp_name)
    #Merge all company follower profiles in one final dataframe
    cleaned_df = mergeRawDataFrame(company_followerProfiles)
    #Give the columns of the new dataframe the desired column names.
    cleaned_df = cleanRawDataFrame(cleaned_df)
    total_userIDS = len(set([value for value in cleaned_df['userID']]))
    print("{} - {}".format(comp_name, total_userIDS))
    return 

for comp in sport_companies:
    check_IDS(comp)

/Users/nielsvangorsel/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-5-c29f81a2966e>:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([df, temp_df], ignore_index=True)


plutosport.nl - 697


/Users/nielsvangorsel/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (88) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/nielsvangorsel/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (89) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


We have a double name: mishaa_79_
voetbalshopnl - 1786


/Users/nielsvangorsel/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (0,1,3,6,8,11,12,13,14,20,22,24,26,31,39,40,41,42,43,46,47,75,76,89,91) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

/Users/nielsvangorsel/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (0,2,6,7,8,12,13,14,16,18,19,21,26,28,29,30,32,33,44,45,88,89,90,91,93,95) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


soccerfanshop - 1913


/Users/nielsvangorsel/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/nielsvangorsel/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (0,7,11,12,13,14,16,18,19,21,29,30,31,32,33,50,88,89,90,91,93,95) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


jdsportsnl - 1980
decathlonnederland - 2719
gorillasportsnl - 1059


KeyboardInterrupt: 